In [ ]:
#https://jovian.ai/droste-benedikt/02-article-pytorch-multilabel-classification-v2/v/1?utm_source=embed#C21

In [1]:
import pandas as pd 
import numpy as np
import glob
from tqdm import tqdm
import cv2
from sklearn.model_selection import train_test_split
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import datasets, models, transforms
from torchvision.models import resnet18

from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [2]:
#Coloring for print outputs
class color:
   RED = '\033[91m'
   BOLD = '\033[1m'
   END = '\033[0m'

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
signs = ['3.24', '1.16', '5.15.5', '5.19.1', '5.19.2', '1.20.1', '8.23',
'2.1', '4.2.1', '8.22.1', '6.16', '1.22', '1.2', '5.16', '3.27',
'6.10.1', '8.2.4', '6.12', '5.15.2', '3.13', '3.1', '3.20', '3.12',
'7.14.2', '5.23.1', '2.4', '5.6', '4.2.3', '8.22.3', '5.15.1',
'7.3', '3', '2.3.1', '3.11', '6.13', '5.15.4', '8.2.1', '1.34.3',
'8.2.2', '5.15.3', '1.17', '4.1.1', '4.1.4', '3.25', '1.20.2',
'8.22.2', '6.9.2', '3.2', '5.5', '5.15.7', '7.12', '8.2.3',
'5.24.1', '1.25', '3.28', '5.9.1', '5.15.6', '8.1.1', '1.10',
'6.11', '3.4', '6.10', '6.9.1', '8.2.5', '5.15', '4.8.2', '8.22',
'5.21', '5.18']

In [5]:
len(signs)

69

In [6]:
N_SIGN = 70
BATCH_SIZE=2

## Датасет

Прежде чем разбираться с моделями, нам надо в первую очередь разобраться с тем, как грузить датасет. Давайте напишем класс в торче для этого.

In [83]:
class ImageDataset(Dataset):
    def __init__(self, data_df, type_df='train', transform=None):

        self.data_df = data_df
        self.type_df = type_df
        self.transform = transform

    def __getitem__(self, idx):
        # достаем имя изображения и ее лейблы
        image_name = self.data_df.iloc[idx]['img']
        type_df = self.type_df
        label1 = self.data_df.iloc[idx]['sign1']
        label2 = self.data_df.iloc[idx]['sign2']
        label3 = self.data_df.iloc[idx]['sign3']
        label4 = self.data_df.iloc[idx]['sign4']
        label5 = self.data_df.iloc[idx]['sign5']
        label6 = self.data_df.iloc[idx]['sign6']
        label7 = self.data_df.iloc[idx]['sign7']
        label8 = self.data_df.iloc[idx]['sign8']

        # читаем картинку. read the image
        image = cv2.imread(f"./content/{type_df}/{image_name}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        
        # преобразуем, если нужно
        if self.transform:
            image = self.transform(image)

        sample = {'image':image, 'labels': {'label_sign1':label1, 'label_sign2':label2, 'label_sign3':label3, 'label_sign4':label4,
                                            'label_sign5':label5, 'label_sign6':label6, 'label_sign7':label7, 'label_sign8':label8}}
        return sample   
            
    def __len__(self):
        return len(self.data_df)

In [84]:
# задаем преобразование изображения.

train_transform = transforms.Compose([
    transforms.Resize(256),
    #transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225]),
])

valid_transform = transforms.Compose([
    transforms.Resize(256),
    #transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225]),
])

In [85]:
# читаем датасет
data_df = pd.read_csv("./content/train.csv")

In [86]:
data_df.head(3)

,id,img,sing1,sing2,sing3,sing4,sing5,sing6,sing7,sing8
0,807,5-avi-frame24_jpg.rf.5dec372f9195e9a88ff7dd3bd...,1,2,37,0,0,0,0,0
1,121,6-avi-frame6431_jpg.rf.1ad48ac0ce545b88cefb946...,8,0,0,0,0,0,0,0
2,1130,9-avi-frame1457_jpg.rf.634a979898a9caa4d106913...,21,0,0,0,0,0,0,0


In [87]:
test_df = pd.read_csv("./content/test.csv")

In [88]:
len(test_df)

388

In [63]:
test_df['sign1'] = 0
test_df['sign2'] = 0
test_df['sign3'] = 0
test_df['sign4'] = 0
test_df['sign5'] = 0
test_df['sign6'] = 0
test_df['sign7'] = 0
test_df['sign8'] = 0

In [64]:
test_df.head()

,id,img,sign1,sign2,sign3,sign4,sign5,sign6,sign7,sign8
0,754,6-avi-frame14887_jpg.rf.bb0bf6b4b122c23e1b33a9...,0,0,0,0,0,0,0,0
1,29,5-avi-frame2916_jpg.rf.1ecdbbc129d33896fd25b9b...,0,0,0,0,0,0,0,0
2,1157,5-avi-frame2871_jpg.rf.f73998176f8a19ee03f8704...,0,0,0,0,0,0,0,0
3,1049,6-avi-frame5752_jpg.rf.a067b0fc55b770c9b10bb7a...,0,0,0,0,0,0,0,0
4,715,6-avi-frame5678_jpg.rf.f140419d224703d49fe65db...,0,0,0,0,0,0,0,0


In [65]:
from os import listdir

print("Обучающей выборки " ,len(listdir("./content/train")))
print("Тестовой выборки " ,len(listdir("./content/test")))

Обучающей выборки  778
Тестовой выборки  388


In [66]:
# разделим датасет на трейн и валидацию, чтобы смотреть на качество
train_df, valid_df = train_test_split(data_df, test_size=0.2, random_state=43)

In [67]:
train_df.shape, valid_df.shape

((622, 10), (156, 10))

In [68]:
train_dataset = ImageDataset(train_df, transform=train_transform)
valid_dataset = ImageDataset(valid_df, transform=valid_transform)
test_dataset = ImageDataset(test_df, type_df='test', transform=valid_transform)

In [69]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True, num_workers=0)
valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset, batch_size=BATCH_SIZE, pin_memory=True, num_workers=0)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=1, pin_memory=True, num_workers=0)

## Вспомогательные функции

## Модель

In [24]:
class MultilabelClassifier(nn.Module):
    def __init__(self, n_sign1, n_sign2, n_sign3, n_sign4, n_sign5, n_sign6, n_sign7, n_sign8):
        super().__init__()
        self.resnet = models.resnet34(pretrained=True)
        #self.resnet = models.resnet34(weights=ResNet34_Weights.IMAGENET1K_V1)
        
        self.model_wo_fc = nn.Sequential(*(list(self.resnet.children())[:-1]))

        self.sign1 = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=512, out_features=N_SIGN)
        )
        self.sign2 = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=512, out_features=N_SIGN)
        )
        self.sign3 = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=512, out_features=N_SIGN)
        )
        self.sign4 = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=512, out_features=N_SIGN)
        )
        self.sign5 = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=512, out_features=N_SIGN)
        )
        self.sign6 = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=512, out_features=N_SIGN)
        )
        self.sign7 = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=512, out_features=N_SIGN)
        )
        self.sign8 = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=512, out_features=N_SIGN)
        )
        

    def forward(self, x):
        x = self.model_wo_fc(x)
        x = torch.flatten(x, 1)

        return {
            'sign1': self.sign1(x),
            'sign2': self.sign2(x),
            'sign3': self.sign3(x),
            'sign4': self.sign4(x),
            'sign5': self.sign5(x),
            'sign6': self.sign6(x),
            'sign7': self.sign7(x),
            'sign8': self.sign8(x),
        }

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MultilabelClassifier(70,70,70,70,70,70,70,70).to(device)

O:\!P\!P00_AI_hack_ai\hack_ai_venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
O:\!P\!P00_AI_hack_ai\hack_ai_venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [26]:
def criterion(loss_func,outputs,pictures):
    losses = 0
    for i, key in enumerate(outputs):
        losses += loss_func(outputs[key], pictures['labels'][f'label_{key}'].to(device))
    return losses

In [190]:
def training(model, device, lr_rate, epochs, train_loader):
    num_epochs = epochs
    losses = []
    checkpoint_losses = []

    optimizer = torch.optim.Adam(model.parameters(), lr=lr_rate)
    n_total_steps = len(train_loader)

    loss_func = nn.CrossEntropyLoss()

    for epoch in tqdm(range(num_epochs)):
        for i, pictures in enumerate(train_loader):
            images = pictures['image'].to(device)
            pictures = pictures

            outputs = model(images)
            #print(outputs)

            loss = criterion(loss_func,outputs, pictures)
            losses.append(loss.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i+1) % (int(n_total_steps/10)) == 0:
                checkpoint_loss = torch.tensor(losses).mean().item()
                checkpoint_losses.append(checkpoint_loss)
                print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {checkpoint_loss:.4f}')
                torch.save(model, f'./content/model{i+1}.pth')
    return checkpoint_losses

checkpoint_losses = training(model,device,0.0001,10,train_loader)

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

Epoch [1/10], Step [31/311], Loss: 6.8182
Epoch [1/10], Step [62/311], Loss: 7.6634
Epoch [1/10], Step [93/311], Loss: 7.4110
Epoch [1/10], Step [124/311], Loss: 7.4675
Epoch [1/10], Step [155/311], Loss: 7.3504
Epoch [1/10], Step [186/311], Loss: 7.1963
Epoch [1/10], Step [217/311], Loss: 7.0202
Epoch [1/10], Step [248/311], Loss: 6.8979
Epoch [1/10], Step [279/311], Loss: 7.0220
Epoch [1/10], Step [310/311], Loss: 7.0084


 10%|████████                                                                        | 1/10 [08:15<1:14:20, 495.60s/it]

Epoch [2/10], Step [31/311], Loss: 7.1377
Epoch [2/10], Step [62/311], Loss: 7.0071
Epoch [2/10], Step [93/311], Loss: 7.0273
Epoch [2/10], Step [124/311], Loss: 6.9573
Epoch [2/10], Step [155/311], Loss: 6.9450
Epoch [2/10], Step [186/311], Loss: 6.8155
Epoch [2/10], Step [217/311], Loss: 6.7890
Epoch [2/10], Step [248/311], Loss: 6.7263
Epoch [2/10], Step [279/311], Loss: 6.6315
Epoch [2/10], Step [310/311], Loss: 6.6226


 20%|████████████████                                                                | 2/10 [16:17<1:05:00, 487.55s/it]

...

 80%|█████████████████████████████████████████████████████████████████▌                | 8/10 [58:41<13:25, 402.59s/it]

Epoch [9/10], Step [31/311], Loss: 4.7790
Epoch [9/10], Step [62/311], Loss: 4.7512
Epoch [9/10], Step [93/311], Loss: 4.7286
Epoch [9/10], Step [124/311], Loss: 4.7111
Epoch [9/10], Step [155/311], Loss: 4.6921
Epoch [9/10], Step [186/311], Loss: 4.6648
Epoch [9/10], Step [217/311], Loss: 4.6412
Epoch [9/10], Step [248/311], Loss: 4.6227
Epoch [9/10], Step [279/311], Loss: 4.6035
Epoch [9/10], Step [310/311], Loss: 4.5911


 90%|████████████████████████████████████████████████████████████████████████        | 9/10 [1:04:43<06:29, 389.77s/it]

Epoch [10/10], Step [31/311], Loss: 4.5662
Epoch [10/10], Step [62/311], Loss: 4.5456
Epoch [10/10], Step [93/311], Loss: 4.5183
Epoch [10/10], Step [124/311], Loss: 4.5000
Epoch [10/10], Step [155/311], Loss: 4.4726
Epoch [10/10], Step [186/311], Loss: 4.4463
Epoch [10/10], Step [217/311], Loss: 4.4272
Epoch [10/10], Step [248/311], Loss: 4.4025
Epoch [10/10], Step [279/311], Loss: 4.3918
Epoch [10/10], Step [310/311], Loss: 4.3855


100%|███████████████████████████████████████████████████████████████████████████████| 10/10 [1:10:39<00:00, 423.93s/it]


## Посмотрим метрики нашей итоговой модели на валидации.

In [70]:
model = torch.load(f'./content/model310.pth')

In [31]:
def validation(model, dataloader, *args):

    all_predictions = torch.tensor([]).to(device)
    all_true_labels = torch.tensor([]).to(device)

    with torch.no_grad():
        n_correct = []
        n_class_correct = []
        n_class_samples = []
        n_samples = 0

        for arg in args:
            n_correct.append(len(arg))
            n_class_correct.append([0 for i in range(len(arg))])
            n_class_samples.append([0 for i in range(len(arg))])
        #print(f'n_correct={n_correct}')

        for pictures in dataloader:
            images = pictures['image'].to(device)
            outputs = model(images)
            #print(f'outputs={outputs}')        
            labels = [pictures['labels'][picture].to(device) for picture in pictures['labels']]
            #print(f'labels={labels}')

            for i,out in enumerate(outputs):
                _, predicted = torch.max(outputs[out],1)
                #print(f'predicted={predicted}')
                #print(f'labels[{i}]={labels[i]}, len(labels[i])={len(labels[i])}')
                n_correct[i] += (predicted == labels[i]).sum().item()
                #print(f'n_correct[{i}]={n_correct[i]}')

                if i == 0:
                    n_samples += (labels[i].size(0))

                for k in range(BATCH_SIZE): # 16
                    label = labels[i][k]
                    pred = predicted[k]
                    if (label == pred):
                        n_class_correct[i][label] += 1
                    n_class_samples[i][label] += 1
          
    return n_correct, n_samples, n_class_correct, n_class_samples

def class_acc(n_correct,n_samples,n_class_correct,n_class_samples,class_list):
    for i in range(len(class_list)):
        print("-------------------------------------------------")
        acc = 100.0 * n_correct[i] / n_samples
        print(color.BOLD + color.RED + f'Overall class performance: {round(acc,1)} %' + color.END)
        for k in range(len(class_list[i])):
            acc = 100.0 * n_class_correct[i][k] / n_class_samples[i][k]
            print(f'Accuracy of {class_list[i][k]}: {round(acc,1)} %')
    print("-------------------------------------------------")

classes_sign1 = [x for x in range(N_SIGN)]
classes_sign2 = [x for x in range(N_SIGN)]
classes_sign3 = [x for x in range(N_SIGN)]
classes_sign4 = [x for x in range(N_SIGN)]
classes_sign5 = [x for x in range(N_SIGN)]
classes_sign6 = [x for x in range(N_SIGN)]
classes_sign7 = [x for x in range(N_SIGN)]
classes_sign8 = [x for x in range(N_SIGN)]
class_list = [classes_sign1, classes_sign2, classes_sign3, classes_sign4, classes_sign5, classes_sign6, classes_sign7, classes_sign8]

n_correct, n_samples, n_class_correct, n_class_samples = validation(model, valid_loader, classes_sign1, classes_sign2, classes_sign3, classes_sign4, classes_sign5, classes_sign6, classes_sign7, classes_sign8)

class_acc(n_correct,n_samples,n_class_correct,n_class_samples,class_list)

-------------------------------------------------
Overall class performance: 121.2 %


ZeroDivisionError: float division by zero

In [77]:
test_loader

In [90]:
model.eval()
valid_predicts = []

for i, pictures in tqdm(enumerate(test_loader)):
    images = pictures['image'].to(device)
    outputs = model(images)
    #print(f'pred={pred}')
    
    sample_predict = []
    for _, out in enumerate(outputs):
        _, predicted = torch.max(outputs[out],1)
        #print(f'predicted.numpy()[0]={predicted.numpy()[0]}')
        sample_predict.append(predicted.numpy()[0])
        #if predicted == labels[i]
    #print(f'sample_predict={sample_predict}')    
    valid_predicts.append(sample_predict)


1it [00:00,  3.07it/s]

0


2it [00:00,  3.40it/s]

1


3it [00:00,  3.75it/s]

...

386it [01:35,  3.98it/s]

386


388it [01:35,  4.13it/s]

387


388it [01:35,  4.05it/s]


In [92]:
len(valid_predicts)

388

In [93]:
test_df[['sign1', 'sign2', 'sign3', 'sign4','sign5','sign6', 'sign7', 'sign8']] = valid_predicts

In [96]:
cols = ['id', 'sing1', 'sing2', 'sing3', 'sing4', 'sing5', 'sing6', 'sing7', 'sing8']

In [97]:
df_submission = test_df[['id', 'sign1', 'sign2', 'sign3', 'sign4','sign5','sign6', 'sign7', 'sign8']]
df_submission.columns = cols
df_submission.to_csv("./content/RSF_solution.csv", index=False)